## Планирование движения

В данном домашнем задании вам необходимо реализовать свой планер движения. Задача полученного планера -- проехать в симуляции предложенные ниже сценарии. Чем больше сценариев проедет ваш планер, тем выше балл за домашнее задание вы получите.

#### Установка зависимостей

In [1]:
%pip install jsonpickle dacite shapely

Note: you may need to restart the kernel to use updated packages.


#### Запуск сервера

Для каждого запуска ядра ноутбука нужно исполнить эту ячейку один раз.

In [2]:
import py_planning
py_planning.init()

def display_simulator(case_name):
    from IPython.display import IFrame, display
    url = 'http://127.0.0.1:8008?scenario=' + case_name
    print("You can also open in browser: " + url)
    display(IFrame(url, width="100%", height=650))

### Пример планера

Чтобы реализовать планер, вам необходимо написать функцию, которая на вход принимает текущее состояние автомобиля и окружающего мира (`State`), а выдает план движения автомобиля (`PlannedState`). Подробно с устройством этих сущностей вы можете ознакомиться в файле `py_planning/data_types.py`.

Симулятор итеративно обновляет состояние автомобиля `State`, подаваемое на вход функции планирования, при помощи плана движения, который функция планирования вернула на предыдущей итерации.

Ниже для примера представлен простейший планер. Функция `do_simple_plan` реализует алгоритм, который планирует движение вдоль центра дороги с постоянной скоростью.

In [3]:
from py_planning.data_types import PlannedPath, PlannedState, State, Position  # data types used by planner interface
from shapely.geometry import LineString, Point

import time

"""
find closest point on a polyline to the given point
"""
def get_index_of_closest_point(line: LineString, point: Point):
    closest_point_index = None
    min_distance = float('inf')

    for i, line_point in enumerate(line.coords):
        line_point = Point(line_point)
        distance = point.distance(line_point)
        if distance < min_distance:
            min_distance = distance
            closest_point_index = i

    return closest_point_index

"""
This function is called by the simulator for each tick.
It should return recent planned trajectory up to date with the environment state.
'state' parameter contains current world observations and vehicle state.
"""
def do_simple_plan(state: State) -> PlannedPath:
    vehicle_pose = state.vehicle_pose
    vehicle_pos = Point(vehicle_pose.pos.x, vehicle_pose.pos.y)  # current position of the AV

    centerline = LineString([(p.x, p.y) for p in state.lane_path.centerline])

    closest_index = get_index_of_closest_point(centerline, vehicle_pos) + 10
    max_poses_count = 70

    # as a baseline here we just follow the centerline
    planned_states = [
        PlannedState(pos=p) for p in state.lane_path.centerline
    ][closest_index:closest_index+ max_poses_count]

    # we have to start trajectory close to ego's current position
    planned_states = [PlannedState(pos=Position(vehicle_pos.x, vehicle_pos.y))] + planned_states
    print(planned_states)

    return PlannedPath(states=planned_states)

Чтобы запустить симуляцию алгоритма `do_simple_plan` на сценарии `rough-road`, выполните следующую ячейку и нажмите `Play`. 

Симуляция должна завершиться с ошибкой `Case failed: Collision with static object` из-за столкновения со статическим препятствием. Также в выводе планера вы увидите время работы и число совершенных тиков планирования (сколько раз запускалась функция `do_simple_plan`).

In [4]:
display_simulator('rough-road')

# run the case in the simulator, watch the visualization
py_planning.run_planner(
    do_simple_plan,
    stop_on_fail=True  # set to False to continue planning after case fail (useful for debugging, can restart scenario in simulator)
)

You can also open in browser: http://127.0.0.1:8008?scenario=rough-road


Case started


KeyboardInterrupt: 

## Задание

Реализуйте функцию `do_plan` и проверьте ее работу на пяти сценариях, предложенных ниже.

**ВАЖНО!!!** Перед отправкой ноутбука запустите команду `Kernel -> Restart Kernel and Run All Cells` и последовательно запустите симуляцию сценариев, сохраните и отправьте ноутбук вместе с полученными выходами ячеек. Обратите внимание, что **при проверке запускаться и оцениваться будут только те сценарии, которые отмечены, как успешно пройденные**, то есть вывод ячейки содержит строку `Congrats! Case completed successfully`.

В каждом сценарии вам нужно будет доехать до конца дороги, уложившись в лимит по времени, так, чтобы:
- не задеть статических и динамических препятствий;
- не выехать за пределы дороги;
- не превысить скоростной лимит.

In [5]:
def do_plan(state: State) -> PlannedPath:
    return PlannedPath(states=[PlannedState(pos=state.vehicle_pose.pos, velocity=0) for i in range(10)])

#### Сценарий №1 (2 балла)

В данном сценарии вам необходимо объехать статические препятствия по извилистой дороге. Для этого нужно реализовать планер геометрии. Например, вы можете воспользоваться графовым планером, который мы обсуждали в пятой лекции (также тут будут полезны пятый и шестой семинары), либо доработать логику из функции `do_simple_plan` эвристически. Также в своем решении вы можете использовать любые другие подходы.

Ограничение по времени на прохождение сценария: 25 секунд.

In [6]:
display_simulator('rough-road')

# run the case in the simulator, watch the visualization
py_planning.run_planner(
    do_plan,
    stop_on_fail=True  # set to False to continue planning after case fail (useful for debugging, can restart scenario in simulator)
)

You can also open in browser: http://127.0.0.1:8008?scenario=rough-road


Case started


KeyboardInterrupt: 

#### Сценарий №2 (2 балла)

В данном сценарии вам так же необходимо объехать статические препятствия, как и в пердыдущем.

Ограничение по времени на прохождение сценария: 20 секунд.

In [7]:
display_simulator('avoid-static')

# run the case in the simulator, watch the visualization
py_planning.run_planner(
    do_plan,
    stop_on_fail=True  # set to False to continue planning after case fail (useful for debugging, can restart scenario in simulator)
)

You can also open in browser: http://127.0.0.1:8008?scenario=avoid-static


Case started


KeyboardInterrupt: 

#### Сценарий №3 (2 балла)

В данном сценарии вам необходимо пропустить пешеходов на пешеходном переходе. Здесь необходимо реализовать уже планер скорости. Это также можно сделать, например, при помощи графового планера, либо при помощи эвристик. Также в своем решении вы можете использовать любые другие подходы.

Ограничение по времени на прохождение сценария: 45 секунд.

In [ ]:
display_simulator('crosswalks')

# run the case in the simulator, watch the visualization
py_planning.run_planner(
    do_plan,
    stop_on_fail=True  # set to False to continue planning after case fail (useful for debugging, can restart scenario in simulator)
)

You can also open in browser: http://127.0.0.1:8008?scenario=crosswalks


Case started


#### Сценарий №4 (2 балла)

В данном сценарии вам необходимо обогнать два медленных автомобиля. 

Ограничение по времени на прохождение сценария: 35 секунд.

In [ ]:
display_simulator('two-car-overtake')

# run the case in the simulator, watch the visualization
py_planning.run_planner(
    do_plan,
    stop_on_fail=True  # set to False to continue planning after case fail (useful for debugging, can restart scenario in simulator)
)

#### Сценарий №5 (2 балла)

В данном сценарии вам необходимо встроиться в медленный поток автомобилей.

Ограничение по времени на прохождение сценария: 90 секунд.

In [ ]:
display_simulator('merging')

# run the case in the simulator, watch the visualization
py_planning.run_planner(
    do_plan,
    stop_on_fail=True  # set to False to continue planning after case fail (useful for debugging, can restart scenario in simulator)
)